In [1]:
import csv
import copy
import math
import pandas as pd
from datetime import datetime

In [2]:
# 1 — remove_missing
def remove_missing(records, fields):
    """
    this function removes the records that have missing data in certain fields that can't be empty
    since there are some fields that are okay to be empty :)
    + take a note that this function is pure 100% since it does not affect the original dataset :)
    """
    return list(filter(
        lambda record: all(record.get(field) not in (None, "", "NA") for field in fields),
        records
    ))


# 2 — fill_missing
def fill_missing(records, defaults):
    """
    this function fills missing fields in records with their default value in `defaults` dictionary
    sounds confusing but easy tho :)
    """
    def is_missing(value):
        return value in (None, "", "NA") or (isinstance(value, float) and math.isnan(value))
    
    return list(map(
        lambda record: {
            k: defaults.get(k) if is_missing(v) else (bool(v) if isinstance(defaults.get(k), bool) else v)
            for k, v in record.items()
        },
        records
    ))


# 3 — standardize_date
def standardize_date(record, field):
    """
    this function standardizes the date of the given field of the given record (exactly one record)
    """
    new_record = copy.deepcopy(record)
    try:
        parsed = datetime.strptime(new_record[field], "%m/%d/%Y")
        new_record[field] = parsed.strftime("%Y-%m-%d")
    except:
        pass
    return new_record


# 4 — standardize_dates
def standardize_dates(records, field):
    """
    this function does the same as the previous function but for multiple records
    """
    return list(map(lambda record: standardize_date(record, field), records))


# 5 — normalize_numbers
def normalize_numbers(records, fields, precision):
    """
    this function rounds the values of the given records to a fixed number of decimals (according to the given precision)
    """
    return list(map(
        lambda record: {k: (round(float(value), precision) if k in fields else value) for k, value in record.items()},
        records
    ))


# 6 — clean_strings
def clean_strings(records, fields):
    """
    this function removes leading and trailing spaces then converts the text to 'title case' to look clean :)
    """
    return list(map(
        lambda record: {k: (value.strip().title() if k in fields else value) for k, value in record.items()},
        records
    ))

In [3]:
# CSV Loader and Saver
def load_csv(filename):
    return pd.read_csv(filename)


def save_csv(filename, records):
    if not records:
        return
    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=records[0].keys())
        writer.writeheader()
        writer.writerows(records)

In [7]:
copy_of_original_dataset = load_csv("retail_store_sales.csv")
copy_of_original_df = pd.DataFrame(copy_of_original_dataset)
copy_of_original_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,1905,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False


In [8]:
len(copy_of_original_df)

12575

In [9]:
# let's check the number of empty values in fields
copy_of_original_df.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [18]:
# replace NaN with None for consistency
copy_of_original_df = copy_of_original_df.where(pd.notna(copy_of_original_df), None)
# changing the dataframe into a dictionary to be proccessed by the functions
copy_of_original_records = copy_of_original_df.to_dict(orient="records")


In [25]:
"""
testing remove_missing() function :) 
"""
removed_missing_records= remove_missing(copy_of_original_records,
['Item', 'Price Per Unit', 'Quantity', 'Total Spent', 'Discount Applied'])
# changing the dictionary back into dataframe
removed_missing_df = pd.DataFrame(removed_missing_records)
removed_missing_df.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [26]:
removed_missing_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,1905,False
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False
4,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,6/10/2023,True


In [27]:
len(removed_missing_df)

7579

In [30]:
"""
testing fill_missing() function :) 
"""

defaults = {
    "Item": "Unknown",
    "Price Per Unit": 0,
    "Quantity": 0,
    "Total Spent": 0,
    "Discount Applied": False
}
filled_missing_records = fill_missing(copy_of_original_records, defaults)
filled_missing_df = pd.DataFrame(filled_missing_records)
filled_missing_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,1905,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,False
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False


In [31]:
len(filled_missing_df)

12575

In [32]:
filled_missing_df.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [33]:
"""
testing standardize_dates() function :) 
"""
standardized_records = standardize_dates(copy_of_original_records, "Transaction Date")
standardize_df = pd.DataFrame(standardized_records)
standardize_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,1905,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,None
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [51]:
"""
testing normalize_numbers() function :) 
"""
normalized_numbers_records = normalize_numbers(copy_of_original_records, ['Price Per Unit', 'Total Spent'], 0)
normalized_numbers_df = pd.DataFrame(normalized_numbers_records)
normalized_numbers_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.0,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,22.0,2.0,43.0,Credit Card,Online,1905,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,28.0,9.0,248.0,Credit Card,Online,5/7/2022,None
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.0,7.0,88.0,Digital Wallet,Online,10/2/2022,False


In [38]:
"""
testing clean_strings() function :) 
"""
cleaned_string_records = clean_strings(copy_of_original_records, ['Category', 'Payment Method'])
cleaned_string_df = pd.DataFrame(cleaned_string_records)
cleaned_string_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,1905,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,None
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False


In [39]:
"""
testing save_csv() function :) 
"""
save_csv("removed_missing_data.csv", removed_missing_records)

In [41]:
created_dataset = load_csv("removed_missing_data.csv")
created_df = pd.DataFrame(created_dataset)
len(created_df)

7579